In [ ]:
import cv2 as cv
import numpy as np
import os
from matplotlib import pyplot as plt


def matching_rgb(item_no, output_image_path, threshold = 0.8):
    # Define the paths for the main image and the template
    main_image_path = '1.jpg'
    item_image_path = f'items_list/item{item_no}.jpg'  # Assuming items are named as item1.png, item2.png, etc.
    
    # Load the main image and convert it to grayscale
    img_rgb = cv.imread(main_image_path)
    assert img_rgb is not None, "Main image could not be read, check the path."
    img_gray = cv.cvtColor(img_rgb, cv.COLOR_BGR2GRAY)
    
    # Load the template (item) image
    template = cv.imread(item_image_path, cv.IMREAD_UNCHANGED)
    assert template is not None, f"Item image 'item{item_no}.jpg' could not be read, check the path."
    
    # Create a mask using thresholding to focus on non-white areas
    #_, mask = cv.threshold(gray_item, 220, 255, cv.THRESH_BINARY_INV)
    mask = np.any(template <= 250, axis=2).astype(np.uint8) * 255


    # Find the bounding box of the non-zero mask region
    non_zero_y, non_zero_x = np.nonzero(mask)
    top_left_y, bottom_right_y = np.min(non_zero_y), np.max(non_zero_y)
    top_left_x, bottom_right_x = np.min(non_zero_x), np.max(non_zero_x)

    # Crop the template and the mask to this bounding box
    cropped_template = template[top_left_y:bottom_right_y + 1, top_left_x:bottom_right_x + 1]
    cropped_mask = mask[top_left_y:bottom_right_y + 1, top_left_x:bottom_right_x + 1]

    # Split the main image and cropped template into R, G, B channels
    img_b, img_g, img_r = cv.split(img_rgb)
    template_b, template_g, template_r = cv.split(cropped_template)

    # Define the scales to search through
    scales = np.arange(0.5, 2.01, 0.01)

    # Iterate through each scale and find all matches
    for scale in scales:
        # Resize the cropped template and mask according to the current scale
        scaled_template_r = cv.resize(template_r, (0, 0), fx=scale, fy=scale, interpolation=cv.INTER_LINEAR)
        scaled_template_g = cv.resize(template_g, (0, 0), fx=scale, fy=scale, interpolation=cv.INTER_LINEAR)
        scaled_template_b = cv.resize(template_b, (0, 0), fx=scale, fy=scale, interpolation=cv.INTER_LINEAR)
        scaled_mask = cv.resize(cropped_mask, (0, 0), fx=scale, fy=scale, interpolation=cv.INTER_LINEAR)

        # Skip if the scaled template is larger than the main image
        if scaled_template_r.shape[0] > img_b.shape[0] or scaled_template_r.shape[1] > img_b.shape[1]:
            continue

        # Perform template matching separately for each channel
        res_r = cv.matchTemplate(img_r, scaled_template_r, cv.TM_CCOEFF_NORMED, mask=scaled_mask)
        res_g = cv.matchTemplate(img_g, scaled_template_g, cv.TM_CCOEFF_NORMED, mask=scaled_mask)
        res_b = cv.matchTemplate(img_b, scaled_template_b, cv.TM_CCOEFF_NORMED, mask=scaled_mask)
        
        # Thresholding to find regions where all three match results exceed the threshold
        loc = np.where((res_r >= 0.5) & (res_g >= 0.5) & (res_b >= 0.5) & 
                        ((res_r >= threshold) | (res_g >= threshold) | (res_b >= threshold)))
    
        # Draw rectangles around detected regions
        for pt in zip(*loc[::-1]):
            cv.rectangle(img_rgb, pt, (pt[0] + scaled_mask.shape[1], pt[1] + scaled_mask.shape[0]), (0, 0, 255), 2)
    
    # Save the result image with highlighted matches
    output_image_path = "find_rgb/" + output_image_path + ".jpg"
    cv.imwrite(output_image_path, img_rgb)
    print(f"Result for item {item_no} saved to {output_image_path}")

    # Display the channels side by side
    fig, axs = plt.subplots(2, 3, figsize=(15, 8))
    
    # Display R, G, B channels of the main image
    axs[0, 0].imshow(img_r, cmap='gray')
    axs[0, 0].set_title('Main Image - R Channel')
    axs[0, 0].axis('off')

    axs[0, 1].imshow(img_g, cmap='gray')
    axs[0, 1].set_title('Main Image - G Channel')
    axs[0, 1].axis('off')

    axs[0, 2].imshow(img_b, cmap='gray')
    axs[0, 2].set_title('Main Image - B Channel')
    axs[0, 2].axis('off')
    
    # Display R, G, B channels of the cropped template
    axs[1, 0].imshow(template_r, cmap='gray')
    axs[1, 0].set_title('Template - R Channel')
    axs[1, 0].axis('off')

    axs[1, 1].imshow(template_g, cmap='gray')
    axs[1, 1].set_title('Template - G Channel')
    axs[1, 1].axis('off')

    axs[1, 2].imshow(template_b, cmap='gray')
    axs[1, 2].set_title('Template - B Channel')
    axs[1, 2].axis('off')

    plt.tight_layout()
    plt.show()

# Example usage:
matching_rgb(1, 'find_1') 


In [ ]:
for i in range (1,16):
    matching_rgb(i, f'find_{i}')